<a href="https://colab.research.google.com/github/SpencerPao/Natural-Language-Processing/blob/main/Question%20Answering%20Modeling/Question_Answering_Modeling_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install datasets
!pip install transformers[torch] accelerate -U
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# RoBERTa

In [5]:
from datasets import load_dataset
squad = load_dataset("squad")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset

sampled_train_dataset = squad["train"].shuffle(seed=42).select(range(10000))
train_dataset, val_dataset = train_test_split(sampled_train_dataset, test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict(train_dataset)
val_dataset = Dataset.from_dict(val_dataset)

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [17]:
from transformers import RobertaConfig
config = RobertaConfig.from_pretrained(model_name)
config.hidden_dropout_prob = 0.1 
config.attention_probs_dropout_prob = 0.1  


# GPU TRAINING


In [18]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name, config=config)

In [10]:
import numpy as np

# PERBANDINGAN 1 (PAKAI LOAD_METRICS)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer
import numpy as np
import evaluate

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

accuracy_metric = load_metric("accuracy")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
f1_metric = load_metric("f1")

def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_pred = np.argmax(start_logits, axis=1)
    end_pred = np.argmax(end_logits, axis=1)

    true_start, true_end = p.label_ids
    start_accuracy = accuracy_metric.compute(predictions=start_pred, references=true_start)
    end_accuracy = accuracy_metric.compute(predictions=end_pred, references=true_end)

    start_precision = precision_metric.compute(predictions=start_pred, references=true_start, average='macro')
    end_precision = precision_metric.compute(predictions=end_pred, references=true_end, average='macro')

    start_recall = recall_metric.compute(predictions=start_pred, references=true_start, average='macro')
    end_recall = recall_metric.compute(predictions=end_pred, references=true_end, average='macro')

    start_f1 = f1_metric.compute(predictions=start_pred, references=true_start, average='macro')
    end_f1 = f1_metric.compute(predictions=end_pred, references=true_end, average='macro')

    combined_accuracy = (start_accuracy['accuracy'] + end_accuracy['accuracy']) / 2
    combined_precision = (start_precision['precision'] + end_precision['precision']) / 2
    combined_recall = (start_recall['recall'] + end_recall['recall']) / 2
    combined_f1 = (start_f1['f1'] + end_f1['f1']) / 2

    return {
        "accuracy": combined_accuracy,
        "precision": combined_precision,
        "recall": combined_recall,
        "f1": combined_f1,
    }

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(tokenized_train) * training_args.num_train_epochs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.164500,1.379646,0.743750,0.722973,0.719699,0.707256
2,0.214400,1.161762,0.751500,0.720218,0.718210,0.704070
3,0.252700,1.570169,0.740750,0.720861,0.718160,0.703419
4,0.194100,1.404456,0.747500,0.721226,0.718407,0.704522
5,0.167400,1.616174,0.740000,0.712178,0.706691,0.695015


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall

TrainOutput(global_step=2500, training_loss=0.19861561889648438, metrics={'train_runtime': 3190.1145, 'train_samples_per_second': 12.539, 'train_steps_per_second': 0.784, 'total_flos': 7838902702080000.0, 'train_loss': 0.19861561889648438, 'epoch': 5.0})

# PERBANDINGAN 2 (PAKAI SKLEARN.METRICS)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

accuracy_metric = load_metric("accuracy")

def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_pred = np.argmax(start_logits, axis=1)
    end_pred = np.argmax(end_logits, axis=1)

    true_start, true_end = p.label_ids
    start_accuracy = accuracy_metric.compute(predictions=start_pred, references=true_start)
    end_accuracy = accuracy_metric.compute(predictions=end_pred, references=true_end)

    start_precision, start_recall, start_f1, _ = precision_recall_fscore_support(true_start, start_pred, average='macro', zero_division=1)
    end_precision, end_recall, end_f1, _ = precision_recall_fscore_support(true_end, end_pred, average='macro', zero_division=1)

    combined_accuracy = (start_accuracy['accuracy'] + end_accuracy['accuracy']) / 2
    combined_precision = (start_precision + end_precision) / 2
    combined_recall = (start_recall + end_recall) / 2
    combined_f1 = (start_f1 + end_f1) / 2

    return {
        "accuracy": combined_accuracy,
        "precision": combined_precision,
        "recall": combined_recall,
        "f1": combined_f1,
    }

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(tokenized_train) * training_args.num_train_epochs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.090300,2.240967,0.730500,0.748975,0.746486,0.705343
2,0.114700,2.616011,0.734500,0.748834,0.743430,0.708569
3,0.124400,2.675136,0.717500,0.709403,0.718617,0.675813


TrainOutput(global_step=1500, training_loss=0.10980621337890625, metrics={'train_runtime': 1984.5861, 'train_samples_per_second': 12.093, 'train_steps_per_second': 0.756, 'total_flos': 4703341621248000.0, 'train_loss': 0.10980621337890625, 'epoch': 3.0})

# PERBANDINGAN 3 BATCH SIZE 8, ADDING GRADIENT ACCUMULATION STEPS 2, LOWER LEARNING RATE

In [12]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer, EarlyStoppingCallback, BertForQuestionAnswering
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,

)

accuracy_metric = load_metric("accuracy")

def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_pred = np.argmax(start_logits, axis=1)
    end_pred = np.argmax(end_logits, axis=1)

    true_start, true_end = p.label_ids
    start_accuracy = accuracy_metric.compute(predictions=start_pred, references=true_start)
    end_accuracy = accuracy_metric.compute(predictions=end_pred, references=true_end)

    start_precision, start_recall, start_f1, _ = precision_recall_fscore_support(true_start, start_pred, average='macro', zero_division=1)
    end_precision, end_recall, end_f1, _ = precision_recall_fscore_support(true_end, end_pred, average='macro', zero_division=1)

    combined_accuracy = (start_accuracy['accuracy'] + end_accuracy['accuracy']) / 2
    combined_precision = (start_precision + end_precision) / 2
    combined_recall = (start_recall + end_recall) / 2
    combined_f1 = (start_f1 + end_f1) / 2

    return {
        "accuracy": combined_accuracy,
        "precision": combined_precision,
        "recall": combined_recall,
        "f1": combined_f1,
    }

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(tokenized_train) * training_args.num_train_epochs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.785900,0.653536,0.796250,0.803802,0.805461,0.778986
2,0.522500,0.736596,0.788750,0.791792,0.791530,0.756508
3,0.355300,0.906031,0.769500,0.777048,0.770958,0.733450


TrainOutput(global_step=1500, training_loss=0.5545942891438802, metrics={'train_runtime': 1948.9066, 'train_samples_per_second': 12.315, 'train_steps_per_second': 0.77, 'total_flos': 4703341621248000.0, 'train_loss': 0.5545942891438802, 'epoch': 3.0})

# PERBANDINGAN 4: EVALUATION STRATEGY = STEPS, EVAL STEPS 500, WARMUP STEPS 500

In [13]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer, EarlyStoppingCallback, BertForQuestionAnswering
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",   
    eval_steps=500,               
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    warmup_steps=500,             
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.161900,1.224885,0.770500,0.775615,0.772343,0.741632
1000,0.111200,1.352888,0.768250,0.779850,0.771593,0.734450
1500,0.093100,1.739122,0.765000,0.775004,0.760971,0.718380


TrainOutput(global_step=1500, training_loss=0.12206663767496745, metrics={'train_runtime': 1991.8386, 'train_samples_per_second': 12.049, 'train_steps_per_second': 0.753, 'total_flos': 4703341621248000.0, 'train_loss': 0.12206663767496745, 'epoch': 3.0})

# PERBANDINGAN 5: LOWER LEARNING RATE, LOWER WEIGHT DECAY, INCREASE GRADIENT ACCUMULATION AND WARMUP STEPS, AND INCREASING DROPOUT RATE

In [19]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer, EarlyStoppingCallback, BertForQuestionAnswering
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps", 
    eval_steps=500,
    learning_rate=1e-5,  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.005,  
    gradient_accumulation_steps=4,  
    warmup_steps=1000,           
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,1.110000,0.653521,0.791750,0.835008,0.794790,0.783289


TrainOutput(global_step=750, training_loss=1.1122729899088541, metrics={'train_runtime': 1777.3416, 'train_samples_per_second': 13.503, 'train_steps_per_second': 0.422, 'total_flos': 4703341621248000.0, 'train_loss': 1.1122729899088541, 'epoch': 3.0})

In [ ]:
!zip -r /content/model.zip /content/results/checkpoint-17000

  adding: content/results/checkpoint-17000/ (stored 0%)
  adding: content/results/checkpoint-17000/optimizer.pt
zip I/O error: No space left on device
zip error: Output file write failure (write error on zip file)
